In [0]:
from google.colab import drive
drive.mount('GoogleDrive')

In [0]:
!fusermount -u GoogleDrive

In [0]:
import tensorflow as tf
import numpy as np
import scipy.io as scio
import os

In [19]:
v_feature = scio.loadmat('./My_file_path')
v_feature

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Mon Jul  8 07:31:41 2019',
 '__version__': '1.0',
 'feature': array([[-5.48338913e-02,  8.92851080e-02, -4.73465515e-02, ...,
          0.00000000e+00,  0.00000000e+00,  1.83958597e-05],
        [-5.29286993e-02,  8.88278700e-02, -4.63245265e-02, ...,
          0.00000000e+00,  0.00000000e+00,  1.82688529e-05],
        [-5.97913637e-02,  4.70174461e-02, -7.08648595e-02, ...,
          0.00000000e+00,  1.86215937e-05,  1.86215937e-05],
        ...,
        [-3.04634129e-01, -1.26927431e-01,  2.53682764e-03, ...,
          0.00000000e+00,  0.00000000e+00,  4.80956195e-05],
        [ 9.97678348e-01,  2.62704328e-01,  2.09094800e-01, ...,
          0.00000000e+00,  0.00000000e+00,  4.18415726e-05],
        [-5.80757826e-02,  5.66706719e-02, -5.60402358e-02, ...,
          0.00000000e+00,  0.00000000e+00,  5.06174696e-05]]),
 'label': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,


In [20]:
train_feature = v_feature['feature']
train_feature.shape

(695, 135)

In [21]:
train_label = v_feature['label'].flatten()
train_label.shape

(695,)

In [22]:
train_data = np.c_[train_feature, train_label.reshape(-1, 1)]
train_data.shape

(695, 136)

In [0]:
num_hidden1 = 256 #@param {type: "integer"}
num_hidden2 = 128 #@param {type: "integer"}
initial_rate = 3e-3 #@param {type: "number"}
batch_size = 128 #@param {type: "number"}
num_epoch = 150 #@param {type: "number"}
decay_steps = 30
learning_rate_decay_factor = 0.95
feature_dim = train_feature.shape[1]
file_path = './My_file_path'

In [0]:
graph = tf.Graph()
with graph.as_default():
        
    # learning rate 的选取策略：        
    global_step = tf.Variable(0, name='global_step', trainable=False)
    decay_steps = decay_steps
    learning_rate = tf.train.exponential_decay(learning_rate=initial_rate,
                                                global_step=global_step,
                                                decay_steps=decay_steps,
                                                decay_rate=learning_rate_decay_factor,
                                                staircase=True,
                                                name='exponential_decay')
    
    with tf.name_scope('Input'):
        x = tf.placeholder(tf.float32, shape=[None, feature_dim], name='input')
        label = tf.placeholder(tf.uint8, shape=[None, ], name='label')
        y = tf.one_hot(label, depth=3, dtype=tf.float32)
        keep_prob = tf.placeholder(tf.float32)
    
    with tf.name_scope('Main_Network'):
        with tf.name_scope('FC1'):
            w_1 = tf.get_variable(name='w_fc1', shape=[feature_dim, num_hidden1], initializer=tf.initializers.random_normal(stddev=.1))
            b_1 = tf.get_variable(name='b_fc1', shape=[num_hidden1, ], initializer=tf.initializers.random_normal(stddev=.1))
            layer_1 = tf.nn.relu(tf.matmul(x, w_1) + b_1)

        with tf.name_scope('Dropout1'):
            layer_d1 = tf.nn.dropout(layer_1, keep_prob)

        with tf.name_scope('FC2'):
            w_2 = tf.get_variable(name='w_fc2', shape=[num_hidden1, num_hidden2], initializer=tf.initializers.random_normal(stddev=.1))
            b_2 = tf.get_variable(name='b_fc2', shape=[num_hidden2, ], initializer=tf.initializers.random_normal(stddev=.1))
            layer_2 = tf.nn.relu(tf.matmul(layer_d1, w_2) + b_2)

        with tf.name_scope('Dropout2'):
            layer_d2 = tf.nn.dropout(layer_2, keep_prob)
    
    with tf.name_scope('Output'):
        w_o = tf.get_variable(name='w_fco', shape=[num_hidden2, 3], initializer=tf.initializers.random_normal(stddev=.1))
        b_o = tf.get_variable(name='b_fco', shape=[3, ], initializer=tf.initializers.random_normal(stddev=.1))
        layer_3 = tf.matmul(layer_d2, w_o) + b_o
        y_out = tf.nn.softmax(layer_3)
    
#     # -------------regularization_L2----------------
#     tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_1)
#     tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_2)
#     tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_o)
#     regularizer = tf.contrib.layers.l2_regularizer(scale=1. / 700)
#     reg_tem = tf.contrib.layers.apply_regularization(regularizer)
    
    with tf.name_scope('Loss'):
        cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=layer_3))
#         cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=layer_3) + reg_tem)
    with tf.name_scope('Accuracy'):
        prediction = tf.equal(tf.argmax(layer_3, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(prediction, "float"))
    with tf.name_scope('Train'):
        train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy, global_step=global_step)
    
    tf.summary.scalar('Cross_entropy', cross_entropy, collections=['train'])
    tf.summary.scalar('Accuracy', accuracy, collections=['train'])
#     tf.summary.scalar('global_step', global_step, collections=['train'])
    tf.summary.scalar('learning_rate', learning_rate, collections=['train'])
    tf.summary.histogram('Weights_fc1', w_1, collections=['train'])
    tf.summary.histogram('Biases_fc1', b_1, collections=['train'])

    summ_train = tf.summary.merge_all('train')

In [108]:
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    summ_train_dir = os.path.join(file_path, 'summaries')
    summ_writer = tf.summary.FileWriter(summ_train_dir)
    summ_writer.add_graph(sess.graph)
    
    print('Training ============= (。・`ω´・) ===========\n')
    for epoch in range(num_epoch):
        np.random.shuffle(train_data)
        max_batch = train_data.shape[0] // batch_size
        for num_batch in range(max_batch):
            train_x = train_data[num_batch*batch_size:(num_batch+1)*batch_size, :-1]
            train_l = train_data[num_batch*batch_size:(num_batch+1)*batch_size:, -1]
#         for i in range(3):
#             train_x = train_data[i * 30:(i + 1) * 30, :-1]
#             train_l = train_data[i * 30:(i + 1) * 30, -1]
            _, loss, acc, rt = sess.run([train_op, cross_entropy, accuracy, summ_train], feed_dict={x: train_x, 
                                                                                                    label: train_l, 
                                                                                                    keep_prob: .8})
#         output = sess.run(y_out, feed_dict={x: train_x[0].reshape(-1, feature_dim), 
#                                            label: train_l[0].reshape(-1), 
#                                            keep_prob: 1.})
        summ_writer.add_summary(rt, global_step=epoch)
        print_list = [epoch + 1, loss, acc * 100]
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print('Epoch {0[0]}, cross_entropy: {0[1]:.4f}, accuracy: {0[2]:.2f}%.'.format(print_list))
#             print('Output : {}\n'.format(output))
    print('\nTraining completed.\n')
    loss, acc = sess.run([cross_entropy, accuracy], feed_dict={x: train_feature,
                                                              label: train_label, 
                                                              keep_prob: 1.})
    acc *= 100
    print('Cross_entropy on the whole training set: %.4f, accuracy: %.2f%%.' %(loss, acc))

Training ============= (。・`ω´・) ===========

Epoch 1, cross_entropy: 0.8688, accuracy: 64.84%.
Epoch 10, cross_entropy: 0.2746, accuracy: 89.84%.
Epoch 20, cross_entropy: 0.1436, accuracy: 94.53%.
Epoch 30, cross_entropy: 0.0974, accuracy: 97.66%.
Epoch 40, cross_entropy: 0.0647, accuracy: 98.44%.
Epoch 50, cross_entropy: 0.0174, accuracy: 100.00%.
Epoch 60, cross_entropy: 0.0215, accuracy: 100.00%.
Epoch 70, cross_entropy: 0.0161, accuracy: 99.22%.
Epoch 80, cross_entropy: 0.0323, accuracy: 99.22%.
Epoch 90, cross_entropy: 0.0462, accuracy: 98.44%.
Epoch 100, cross_entropy: 0.0179, accuracy: 100.00%.
Epoch 110, cross_entropy: 0.0072, accuracy: 100.00%.
Epoch 120, cross_entropy: 0.0125, accuracy: 100.00%.
Epoch 130, cross_entropy: 0.0077, accuracy: 100.00%.
Epoch 140, cross_entropy: 0.0119, accuracy: 100.00%.
Epoch 150, cross_entropy: 0.0078, accuracy: 100.00%.

Training completed.

Cross_entropy on the whole training set: 0.0034, accuracy: 100.00%.
